# Notebook – QLoRA_Finetune.ipynb
Parameter‑efficient fine‑tune with 4‑bit base weights + LoRA adapters

## Install dependencies
```bash
pip install transformers peft bitsandbytes trl datasets accelerate
```

In [ ]:
import os, pandas as pd, torch
from datasets import Dataset
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

BASE_MODEL = os.getenv("BASE_MODEL", "mistral-7b-instruct-v0.3")
DATA_FILE = Path("eval_qa50.csv")
OUTPUT_DIR = Path("lora-" + BASE_MODEL.split('/')[-1])
OUTPUT_DIR.mkdir(exist_ok=True)

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg,
    device_map="auto",
    trust_remote_code=True
)

# Build dataset
df = pd.read_csv(DATA_FILE)
def make_prompt(row):
    return f"CONTEXT:\n{row.context}\nQUESTION:\n{row.question}\nANSWER:"
dataset = Dataset.from_dict({
    "prompt": [make_prompt(r) for _, r in df.iterrows()],
    "response": df["answer"].tolist()
})

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="prompt",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

trainer.train()
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Saved LoRA adapters to {OUTPUT_DIR}")
